## Parsing Courses on edx.org

In [67]:
from selenium import webdriver
import time
import pandas as pd

In [68]:
#subjects
subjects = ['Architecture','Art & Culture', 'Biology & Life Sciences', 'Business & Management', 'Chemistry', 'Communication', 'Computer Science', 'Data Analysis & Statistics', 'Design', 'Economics & Finance', 'Education & Teacher Training', 'Electronics', 'Energy & Earth Sciences', 'Engineering', 'Environmental Studies', 'Ethics', 'Food & Nutrition', 'Health & Safety', 'History', 'Humanities', 'Language', 'Law', 'Literature', 'Math', 'Medicine', 'Music', 'Philanthropy', 'Philosophy & Ethics', 'Physics', 'Science', 'Social Sciences']


#course tab on edx.org
base_website = 'https://www.edx.org/search?tab=course'

#query string as per subject
# eg for Chemistry, https://www.edx.org/search?tab=course&subject=Chemistry
website = 'https://www.edx.org/search?tab=course&subject={courseName}'

# maximum number of courses to parse per subject
MAX_COURSES_PER_SUBJECT = 27

In [69]:
# return url for the passed on subject
def subjectUrl(subName):
    subName = subName.replace(" ","%20").replace("&","%26")
    return website.format(courseName=subName)

In [70]:
## NOTE: Configure WebDriver 
driver = webdriver.Edge('../SeleniumDrivers/msedgedriver.exe')

C:\Users\omkar\AppData\Local\Temp/ipykernel_14808/2175063009.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('./SeleniumDrivers/msedgedriver.exe')


### Data 
Description of the data frame fields

| subject |    Subject of course |
|---------|----------------------|
| title |      Course Title |
| desc |       Course Description|
|weeks|      Estimated Weeks required to complete|
|hours|      Average hours per week for completion|
|outcome|    What you'll learn|
|instit|         Hosting Institure|
|level|      Difficulty level of course|
|prereq|     Prerequisite of course|
|lang|       Language|
|url|        URL of course|



In [71]:
subj = []
title = []
desc = []
weeks = []
hours = []
instit = []
level = []
prereq = []
outcome = []
lang = []
url = []

In [72]:
try:
    for subject in subjects:

        #load the subject page
        driver.get(subjectUrl(subject))
        time.sleep(3)

        #Counter cnt
        cnt = 0

        #list of webelements with course links
        course_list_raw = []
        course_list = []

        #do ... while cnt <= MAX_COURSES_PER_SUBJECT and next is enabled
        while(cnt <= MAX_COURSES_PER_SUBJECT):

            iter_list = driver.find_elements_by_xpath("/html//main[@id='main-content']/div[@class='new-search-page search-results']/div[3]/div/div[@class='pgn__data-table-layout-wrapper']//div[@class='row']/div[*]/div[@role='group']/a")

            if ( MAX_COURSES_PER_SUBJECT > cnt + len(iter_list) ):
                #Filter out the urls from elements 
                course_list += list(map(lambda x: str(x.get_attribute('href')), iter_list))
                cnt += len(iter_list)

            else:
                #Filter out the urls from elements 
                course_list += list(map(lambda x: str(x.get_attribute('href')), iter_list[:(MAX_COURSES_PER_SUBJECT-cnt)]))
                cnt += MAX_COURSES_PER_SUBJECT-cnt
            
            # NEXT button on course result page
            NEXT_BTN = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='new-search-page search-results']/div[3]/div//div[@class='pgn__data-table-wrapper']/div[1]/nav/ul[@class='pagination']//button[@class='btn next page-link']")

            # if cnt<MAX_COURSES_PER_SUBJECT and NEXT_BTN is enabled
            if(cnt<MAX_COURSES_PER_SUBJECT and NEXT_BTN.is_enabled()):
                NEXT_BTN.click()
                time.sleep(2)
            else:
                break

        # Subject and count
        print("\nSubject: ", subject)
        print("Count: ",cnt)

        #FOO    
        # for i in course_list:
        #     print(i)  
        
        for course_page in course_list:

            driver.get(course_page)

            tit = driver.find_element_by_css_selector('.col-md-7.pr-4 > h1').text

            des = driver.find_element_by_xpath("//div[@class='p']").text

            wee = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='d-flex flex-column flex-sm-column-reverse']/div[@class='course-snapshot-background']//div[@class='course-snapshot-content py-2 text-primary-500']/div/div[1]/div[@class='ml-3']/div[@class='h4 mb-0']").text
            wee = wee.replace('Estimated ','').replace(' weeks', '')

            hou = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='d-flex flex-column flex-sm-column-reverse']/div[@class='course-snapshot-background']//div[@class='course-snapshot-content py-2 text-primary-500']/div/div[1]/div[@class='ml-3']/div[@class='small']").text
            hou = hou.replace('hours per week','')

            ins = driver.find_element_by_css_selector('li:nth-of-type(1) > .inline-link.muted-link').text

            lev = driver.find_element_by_css_selector('.mb-0.ml-1.pl-3 > li:nth-of-type(3)').text
            lev = lev.replace('Level: ', '')

            pre = driver.find_element_by_css_selector('.mb-0.ml-1.pl-3 p, .mb-0.ml-1.pl-3 div').text

            out = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[3]/div/div[3 or 4]/div[@class='preview-expand-component']/div[2]/div/*").get_attribute('innerHTML')
            
            lan = driver.find_element_by_css_selector('div:nth-of-type(2) > .mb-0.ml-1.pl-3 > li:nth-of-type(1)').text
            lan = lan.replace('Language: ','')

            #print(tit, des, wee, hou, ins, lev, pre, out, lan, sep='\n')


            # append values into list
            subj.append(subject)
            title.append(tit)
            desc.append(des)
            weeks.append(wee)
            hours.append(hou)
            instit.append(ins)
            level.append(lev)
            prereq.append(pre)
            outcome.append(out)
            lang.append(lan)
            url.append(course_page)
            
except:
    print('Process interupted due to an error at url', driver.current_url)



C:\Users\omkar\AppData\Local\Temp/ipykernel_14808/4530444.py:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  iter_list = driver.find_elements_by_xpath("/html//main[@id='main-content']/div[@class='new-search-page search-results']/div[3]/div/div[@class='pgn__data-table-layout-wrapper']//div[@class='row']/div[*]/div[@role='group']/a")
C:\Users\omkar\AppData\Local\Temp/ipykernel_14808/4530444.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  NEXT_BTN = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='new-search-page search-results']/div[3]/div//div[@class='pgn__data-table-wrapper']/div[1]/nav/ul[@class='pagination']//button[@class='btn next page-link']")



Subject:  Art & Culture
Count:  27


C:\Users\omkar\AppData\Local\Temp/ipykernel_14808/4530444.py:52: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tit = driver.find_element_by_css_selector('.col-md-7.pr-4 > h1').text
C:\Users\omkar\AppData\Local\Temp/ipykernel_14808/4530444.py:54: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  des = driver.find_element_by_xpath("//div[@class='p']").text
C:\Users\omkar\AppData\Local\Temp/ipykernel_14808/4530444.py:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  wee = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='d-flex flex-column flex-sm-column-reverse']/div[@class='course-snapshot-background']//div[@class='course-snapshot-content py-2 text-primary-500']/div/div[1]/div[@class='ml-3']/div[@class='h4 mb-0']").text
C:\Users\omkar\AppData\Local\Temp/ipykernel_1

In [ ]:
#closing the browser
driver.close()

In [74]:
df = pd.DataFrame({'subject': subj, 'title': title, 'description': desc, 'weeks': weeks, 'hours': hours, 'institute': instit, 'level':level, 'prerequisites': prereq, 'outcomes': outcome, 'Languages': lang, 'url': url})
df.to_csv('edx-course.csv')
df

,subject,title,description,weeks,hours,institute,level,prerequisites,outcomes,Languages,url
0,Art & Culture,Italian Language and Culture: Beginner (2021-2...,Learn the basics of the Italian language and c...,12,2–5,WellesleyX,Introductory,None,\n<li>How to communicate in a variety of socia...,Italiano,https://www.edx.org/course/italian-language-an...
1,Art & Culture,Steps in Japanese for Beginners1 Part1,Learn basic Japanese expressions to survive in...,5,3–5,WasedaX,Introductory,None,・To say greetings<br>\n・To introduce yourself<...,English,https://www.edx.org/course/steps-in-japanese-f...
2,Art & Culture,Mandarin Chinese Level 1,Mandarin Chinese is rapidly becoming one of th...,6,4–10,MandarinX,Introductory,None,\n<li>Basic spoken Mandarin phrases and vocabu...,English,https://www.edx.org/course/mandarin-chinese-le...
3,Art & Culture,Modern Masterpieces of World Literature,Examine how great modern writers capture the i...,8,3–6,HarvardX,Introductory,None,\n<li>The recent history of World Literature</...,English,https://www.edx.org/course/modern-masterpieces...
4,Art & Culture,Pyramids of Giza: Ancient Egyptian Art and Arc...,"Explore the archaeology, history, art, and hie...",8,2–4,HarvardX,Introductory,None. This is an introductory course.,\n<li>The history and significance of the Giza...,English,https://www.edx.org/course/pyramids-of-giza-an...
5,Art & Culture,Basic Spanish 2: One Step Further,Beginners course for learners of Spanish that ...,4,4–5,UPValenciaX,Introductory,Basic Spanish knowledge or 30 hours of practice,Basic vocabulary to be able to use and underst...,English,https://www.edx.org/course/basic-spanish-2-one...
6,Art & Culture,Comunicación audiovisual desde tu smartphone,Los participantes inscritos en la ruta auditad...,4,2–3,JaverianaX,Introductory,Ninguno,- Las obras creativas y su valor,Español,https://www.edx.org/course/comunicacion-audiov...
7,Art & Culture,Tangible Things: Discovering History Through A...,"Gain an understanding of history, museum studi...",10,1–3,HarvardX,Introductory,None,\n<li>Understanding of museum curation approac...,English,https://www.edx.org/course/tangible-things-dis...
8,Art & Culture,European Paintings: From Leonardo to Rembrandt...,Uncover the meaning behind the art of the grea...,7,2–3,UC3Mx,Introductory,None.,\n<li>Learn to identify paintings by the artis...,English,https://www.edx.org/course/european-paintings-...
9,Art & Culture,Italian Language and Culture: Intermediate (20...,Improve your Italian language skills and expan...,12,3–5,WellesleyX,Intermediate,Secondary school Italian (high school): 1-2 ye...,\n<li>Expand your conversational skills to hav...,Italiano,https://www.edx.org/course/italian-language-an...
